In [1]:
import numpy as np

import os
import cv2

from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


DATA_DIR = os.path.join(os.getcwd(),'data','Smile')

X = []
Y = []

for dir_ in os.listdir(DATA_DIR):
    print(f"Loading files from {dir_}...")
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        image = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (28, 28), interpolation=cv2.INTER_AREA)
        image = np.array(image)

        X.append(image)
        Y.append(dir_)
print("files successfully loaded")

X = np.array(X, dtype="float32") / 255.0
Y = np.asarray(Y)
Y = to_categorical(Y)

(trainX, testX, trainY, testY) = train_test_split(X, Y, test_size=0.20, stratify=Y, random_state=42)

Loading files from 1...


KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Dropout, Flatten, Dense, MaxPool2D
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Input(shape=(28, 28, 1)),

    Conv2D(28, (3, 3), activation="relu", input_shape=(28, 28, 1)),
    MaxPool2D(),
    Conv2D(56, (3, 3), activation="relu"),
    MaxPool2D(),
    Conv2D(112, (3, 3), activation="relu"),
    Flatten(),
    Dense(64, activation="relu"),
    Dense(2, activation="sigmoid"),
])

model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=256, epochs=10, verbose=1)

predictions = model.predict(testX, batch_size=64)

model.save("data\model.keras")

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=['Not Smiling', 'Smiling']))

import matplotlib.pyplot as plt

epochs = range(1, len(history.history["loss"]) + 1)

plt.figure("Plotting of loss / accuracy")
plt.style.use("ggplot")
plt.plot(epochs, history.history["loss"], label="train_loss")
plt.plot(epochs, history.history["val_loss"], label="val_loss")
plt.plot(epochs, history.history["accuracy"], label="train_accuracy")
plt.plot(epochs, history.history["val_accuracy"], label="val_accuracy")
plt.xlabel("Epoch")
plt.ylabel("Loss/Accuracy")
plt.title("Training Loss and Accuracy")
plt.legend()
plt.show()